In [1]:
import Annotator
import gff_helpers
import gffutils
import region_helpers as r
import pybedtools
import os
import pandas as pd
from tqdm import tnrange, tqdm_notebook

In [2]:
GENE_PRIORITY = [
    ['protein_coding','CDS'],
    ['non_coding', 'CDS'],  # shouldn't occur?
    ['protein_coding','5UTR'],
    ['protein_coding','3UTR'],
    ['protein_coding', 'THREE_AND_FIVE_PRIME_UTR'],
    ['protein_coding','intron'],
    ['non_coding','3UTR'],
    ['non_coding','5UTR'],
    ['non_coding', 'THREE_AND_FIVE_PRIME_UTR'],
    ['non_coding','exon'],
    ['non_coding','intron'],
    ['non_coding','transcript'],
    ['non_coding','gene'],
    ['protein_coding','start_codon'],
    ['protein_coding','stop_codon'],
    ['protein_coding','Selenocysteine'],
    ['non_coding','start_codon'],  # shouldn't occur?
    ['non_coding','stop_codon'],  # shouldn't occur?
    ['protein_coding', 'exon'],  # shouldn't occur?
    ['protein_coding', 'transcript'],  # shouldn't occur?
    ['protein_coding', 'gene'],  # shouldn't occur?

    ['non_coding','Selenocysteine'],
]

TRANSCRIPT_PRIORITY = [
    ['protein_coding','CDS'],
    ['protein_coding','5UTR'],
    ['protein_coding','3UTR'],
    ['protein_coding','intron'],
    ['protein_coding', 'THREE_AND_FIVE_PRIME_UTR'],
    ['non_coding','3UTR'],
    ['non_coding','5UTR'],
    ['non_coding', 'THREE_AND_FIVE_PRIME_UTR'],
    ['non_coding','exon'],
    ['non_coding','intron'],
    ['non_coding','transcript'],
    ['non_coding','gene'],
    ['protein_coding','start_codon'],
    ['protein_coding','stop_codon'],
    ['protein_coding','Selenocysteine'],
    ['non_coding','CDS'], # shouldn't occur?
    ['non_coding','start_codon'],  # shouldn't occur?
    ['non_coding','stop_codon'],  # shouldn't occur?
    ['protein_coding','exon'], # shouldn't occur?
    ['protein_coding','transcript'], # shouldn't occur?
    ['protein_coding','gene'], # shouldn't occur?
    ['non_coding','Selenocysteine'],
]


In [3]:
db_file = '/projects/ps-yeolab/genomes/hg19/gencode_v19/gencode.v19.annotation.gtf.db'
a = Annotator.Annotator(db_file)

Initializing/creating defs: 100%|██████████| 8/8 [10:38<00:00, 99.11s/it] 

In [ ]:
# this is an example output from what step1 gives me
bed = '/home/bay001/projects/codebase/bfx/pyscripts/data/WTV5_02_01.basedon_WTV5_02_01.peaks.l2inputnormnew.bed.compressed.chr7.bed.brianannotated2'

In [ ]:
df = pd.read_table(bed, names=['chrom','start','end','pv','fc','strand','annotation'])

In [6]:
HASH_VAL = 1000000
from collections import defaultdict, OrderedDict

def get_all_overlapping_features_from_query(chrom, qstart, qend,
                                            strand):
    """
    Given a query location (chr, start, end), return all features that
    overlap by at least one base. Functions similarly to gffutils db.region(),
    but uses the pre-hashed self.features_dict to greatly speed things up.

    :param chrom : string
    :param qstart : int
    :param qend : int
    :param strand : string
    :return features : list
        list of gffutils.Feature objects.
    """
    features = []
    start_key = int(qstart / HASH_VAL)
    end_key = int(qend / HASH_VAL)
    qstart = qstart + 1 # change 0-based bed to 1-based gff
    qend = qend - 1
    for i in range(start_key, end_key + 1):
        for feature in a.features_dict[chrom, i, strand]:
            if(feature.end == 65419400):
                print("FEATURE: ", feature)
            if qstart <= feature.start and qend >= feature.end:  # feature completely contains query
                features.append(feature)
            elif qstart >= feature.start and qend <= feature.end:  # query completely contains feature
                features.append(feature)
            elif qstart <= feature.start and qend >= feature.start:  # feature partially overlaps (qstart < fstart < qend)
                features.append(feature)
            elif qstart <= feature.end and qend >= feature.end:  # feature partially overlaps (qstart < fend < qend)
                features.append(feature)
    # if qstart == 65419400:
    #     print(features)
    return features
    
def parse_annotation_string(features_string):
    """
    Splits a feature string into a list of feature strings

    :param features_string:
    :return:
    """
    features = features_string.split('|')
    return features

def is_protein_coding(transcript_type):
    """
    if defined protein coding, return True else False

    :param transcript_type:
    :return:
    """
    if transcript_type == 'protein_coding':
        return True
    return False

def return_highest_priority_feature(formatted_features, priority):
    # Build dict
    combined_dict = defaultdict(list)
    for feature_string in formatted_features:
        transcript, start, end, strand, feature_type, gene_id, gene_name, transcript_type_list = feature_string.split(
            ':')
        transcript_type_list = transcript_type_list.split(',')
        for transcript_type in transcript_type_list:
            if is_protein_coding(
                    transcript_type):  # simplify all the types at first
                combined_dict['protein_coding', feature_type].append(
                    feature_string)
            else:
                combined_dict['non_coding',  feature_type].append(
                    feature_string)
    # return the highest one
    combined_dict = OrderedDict(
        combined_dict)  # turn into ordered dict, is that ok?
    combined_dict = sorted(  # sort based on priority list
        combined_dict.iteritems(),
        key=lambda x: priority.index([x[0][0], x[0][1]])
    )
    return combined_dict[0]

def prioritize_transcript_then_gene(formatted_features, gene_priority,
                         transcript_priority):
    unique_transcript_features = defaultdict(list)
    unique_transcripts = defaultdict(list)
    unique_genes = defaultdict(list)
    final = []
    # print("INITIALIZING FUNC")
    for feature_string in formatted_features:
        if feature_string.split(':')[4] != 'gene':
            transcript = feature_string.split(':')[0]
            unique_transcript_features[transcript].append(
                feature_string)

    # Now each unique_transcripts[] contains one record
    # containing a list of features for each unique transcript
    # print("FORMATTED FEATURES: ", formatted_features)
    # print("UNIQUE TRANSCRIPTS: ", unique_transcripts)
    for transcript in unique_transcript_features.keys(): # For each unique transcript
        top_transcript = return_highest_priority_feature(
            unique_transcript_features[transcript],
            transcript_priority
        )[1][0]  # [0] contains the dictionary key
        unique_transcripts[transcript].append(
            top_transcript
        )
        # add gene key
        gene_list = top_transcript.split(':')[5].split(',')
        for gene in gene_list:
            unique_genes[gene].append(top_transcript)
        # print("TRANSCRIPT: {}".format(transcript))
        # print("UNIQUE TRANSCRIPTS[TRANSCRIPT]: ", unique_transcripts[transcript])
        # for each feature, append to each unique gene the top priority feature for each transcript

        """
        for feature_string in unique_transcripts[transcript]:
            # print("T", t)
            genes = feature_string.split(':')[5]
            for gene in genes.split(','):
                unique_genes[gene].append(
                    self.return_highest_priority_feature(
                        unique_transcripts[transcript],
                        transcript_priority
                    )[1][0]  # [0] contains the dictionary key
                )
        """

    for gene, transcripts in unique_genes.iteritems():
        for transcript in transcripts:
            final.append(transcript)
    feature_type, final = return_highest_priority_feature(
        final, gene_priority
    )
    if feature_type[0] == 'non_coding':
        return final[0].replace('exon', 'noncoding_exon').replace('intron', 'noncoding_intron')
    return final[0]

def annotate(
        interval,
        transcript_priority=TRANSCRIPT_PRIORITY,
        gene_priority=GENE_PRIORITY
):
    """
    Given an interval, annotates using the priority

    :param interval:
    :return:
    """
    overlapping_features = get_all_overlapping_features_from_query(
        interval.chrom,
        interval.start,
        interval.end,
        interval.strand
    )
    if len(overlapping_features) == 0:
        return 'intergenic'
    to_append = ''  # full list of genes overlapping features
    transcript = defaultdict(list)
    print(overlapping_features)
    for feature in overlapping_features:  # for each overlapping feature
        for transcript_id in feature.attributes[
            'transcript_id'
        ]:  # multiple genes can be associated with one feature
            transcript[transcript_id].append(
                feature)  # append features to their respective genes
    for transcript, features in transcript.iteritems():
        for feature in features:
            # if 'protein_coding' not in feature.attributes['transcript_type']:
            #     if feature.featuretype == 'exon' or feature.featuretype == 'UTR':
            #         feature.featuretype = 'noncoding_exon'
            if feature.featuretype == 'UTR':
                feature.featuretype = a._classify_utr(feature)
            to_append += "{}:{}:{}:{}:{}:".format(
                transcript,
                feature.start,
                feature.end,
                feature.strand,
                feature.featuretype,
            )
            for t in feature.attributes['gene_id']:
                to_append += '{},'.format(t)
            to_append = to_append[:-1] + ':'
            for t in feature.attributes['gene_name']:
                to_append += '{},'.format(t)
            to_append = to_append[:-1] + ':'
            for t in feature.attributes['transcript_type']:
                to_append += '{},'.format(t)
            to_append = to_append[:-1] + '|'
    to_append = to_append[:-1]
    print("TO APPEND", to_append)
    return prioritize_transcript_then_gene(
        parse_annotation_string(to_append),
        transcript_priority,
        gene_priority
    )

In [ ]:
"""for i in range(15, 30): # for each line (peak)
    features_test = df.ix[i]['features']
    extended_features = []
    for feature in features_test: # for each list of features
        transcript, start, end, strand, feature_type, gene_id, gene_name, transcript_type_list = feature.split(':')
        transcript_types = transcript_type_list.split(',')
        for transcript_type in transcript_types:
            extended_features.append(
                '{}:{}:{}:{}:{}:{}:{}'.format(
                    transcript, start, end, strand, feature_type,
                    gene_id, transcript_type
                )
            )
            print('{}:{}:{}:{}:{}:{}:{}'.format(
                    transcript, start, end, strand, feature_type,
                    gene_id, transcript_type
                ))
    print("____")"""

# Test the priority feature
- given list of features, return with highest priority:

['gene',
 'Selenocysteine',
 'UTR',
 'exon',
 'stop_codon',
 'CDS',
 'start_codon',
 'transcript']

In [5]:

brian_bedhead = ['chrom','start','end','p','f','strand','annotation']
d = '/home/bay001/projects/codebase/bfx/pyscripts/data/'
# bed = os.path.join(d,'WTV5_02_01.basedon_WTV5_02_01.peaks.l2inputnormnew.bed.compressed.chr7.bed.brianannotated2')
bed = os.path.join(d,'ms1_ctrl_r1_01.basedon_ms1_ctrl_r1_01.peaks.l2inputnormnew.bed.compressed.bed')

df = pd.read_table(bed, names=brian_bedhead)
print(df.ix[3082])
annotation_string = parse_annotation_string(df.ix[921]['annotation'])

/home/bay001/anaconda2/envs/brian/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


chrom            chr20
start           257012
end             257036
p                    0
f             0.678857
strand               -
annotation         NaN
Name: 3082, dtype: object


AttributeError: 'numpy.float64' object has no attribute 'split'

In [ ]:
chr7	99965145	99965212

In [ ]:
return_highest_priority_feature(annotation_string, TRANSCRIPT_PRIORITY)

In [ ]:
prioritize_transcript_then_gene(annotation_string, GENE_PRIORITY, TRANSCRIPT_PRIORITY)

In [7]:
interval = pybedtools.create_interval_from_list(['chr7','99965145', '99965212','.','0','+'])
a.prioritize_transcript_then_gene(annotation_string, GENE_PRIORITY, TRANSCRIPT_PRIORITY)

NameError: name 'annotation_string' is not defined

In [8]:
print(interval)
annotate(interval)

chr7	99965145	99965212	.	0	+

[<Feature gene (chr7:99933727-99965454[+]) at 0x2ad8b58c8750>, <Feature transcript (chr7:99933737-99965454[+]) at 0x2ad8b58c8fd0>, <Feature exon (chr7:99964972-99965454[+]) at 0x2ad8b58e18d0>, <Feature transcript (chr7:99933866-99965286[+]) at 0x2ad8b58e7210>, <Feature exon (chr7:99964972-99965286[+]) at 0x2ad8b58ec310>, <Feature gene (chr7:99933737-99965356[+]) at 0x2ad8b58ece50>, <Feature transcript (chr7:99933737-99965355[+]) at 0x2ad8b58ecf10>, <Feature exon (chr7:99964972-99965355[+]) at 0x2ad8b58f6910>, <Feature UTR (chr7:99964998-99965355[+]) at 0x2ad8b590e210>, <Feature transcript (chr7:99949799-99965356[+]) at 0x2ad8b590e2d0>, <Feature exon (chr7:99964972-99965356[+]) at 0x2ad8b590e4d0>, <Feature UTR (chr7:99964998-99965356[+]) at 0x2ad8b590e910>, <Feature transcript (chr7:99949911-99965356[+]) at 0x2ad8b5919510>, <Feature UTR (chr7:99964973-99965356[+]) at 0x2ad8b5928350>, <Feature transcript (chr7:99951576-99965355[+]) at 0x2ad8b5933ed0>, <Featu

'ENST00000432297.2:99965153:99965240:+:5UTR:ENSG00000085514.11:PILRA:protein_coding'

In [ ]:
# OLD FEATURE
import numpy as np
from collections import OrderedDict, defaultdict

def is_protein_coding(transcript_type):
    """
    if defined protein coding, return True else False

    :param transcript_type:
    :return:
    """
    if transcript_type == 'protein_coding':
        return True
    if transcript_type == 'nonsense_mediated_decay':
        return True
    return False

def return_highest_priority_feature(formatted_features, priority):
    """
    Given a list of formatted features, returns the one with the highest priority
    """
    # Build dict
    combined_dict = defaultdict(list)
    for feature_string in formatted_features:
        transcript, start, end, strand, feature_type, gene_id, gene_name, transcript_type_list = feature_string.split(
            ':')
        transcript_type_list = transcript_type_list.split(',')
        for transcript_type in transcript_type_list:
            if is_protein_coding(
                    transcript_type):  # simplify all the types at first
                combined_dict['protein_coding', feature_type].append(
                    feature_string)
            else:
                combined_dict['non_coding', feature_type].append(
                    feature_string)
    # return the highest one
    combined_dict = OrderedDict(
        combined_dict)  # turn into ordered dict, is that ok?
    combined_dict = sorted(  # sort based on priority list
        combined_dict.iteritems(),
        key=lambda x: priority.index([x[0][0], x[0][1]])
    )
    return combined_dict[0]

def prioritize_transcript_then_gene(formatted_features, gene_priority,
                         transcript_priority):
    """
    Group and prioritize for each transcript, then prioritize each #1
    transcript for each gene.

    :param formatted_features:
    :param gene_priority:
    :param transcript_priority:
    :return:
    """
    unique_transcripts = defaultdict(list)
    unique_genes = defaultdict(list)
    final = []

    for feature_string in formatted_features:
        if feature_string.split(':')[4] != 'gene':
            print("appending: {}".format(feature_string))
            unique_transcripts[feature_string.split(':')[0]].append(
                feature_string)
    for transcript in unique_transcripts.keys():  #
        print("parsing transcripts: {}".format(transcript))
        genes = unique_transcripts[transcript][1].split(':')[5]
        for gene in genes.split(','):
            unique_genes[gene].append(
                return_highest_priority_feature(
                    unique_transcripts[transcript],
                    transcript_priority
                )[1][0]  # [0] contains the dictionary key
            )
            print("appended=: {}".format(return_highest_priority_feature(
                    unique_transcripts[transcript],
                    transcript_priority
                )[1][0]))  # [0] contains the dictionary key))
    print("DONE GETTING HI PRIORITY TRANSCRIPTS.")
    print("GENES##############################")
    
    for gene, transcripts in unique_genes.iteritems():
        for transcript in transcripts:
            final.append(transcript)
    print('final', final)
    final = return_highest_priority_feature(
        final, gene_priority
    )[1]
    print("FINAL", final)
    if len(final) > 0:
        return final[0]
    else:
        return 'intergenic'

def parse_annotation_string(features_string):
    """
    Splits a feature string into a list of feature strings

    :param features_string:
    :return:
    """
    features = features_string.split('|')
    return features
